In [ ]:
!pip install pillow numpy


In [3]:
import os
from PIL import Image
import numpy as np

# ==============================================
# 사용자 설정
# ==============================================
IMAGE_FOLDER = "/content/images"         # 이미지 폴더
OUTPUT_BIN = "images_bgr888_concat.bin"  # 합쳐진 바이너리
OUTPUT_H   = "image_index_table.h"       # MCU용 인덱스 테이블 헤더
BASE_ADDR  = 0x71E00000                  # MCU 메모리 시작주소
WIDTH = 800
HEIGHT = 480
BYTES_PER_PIXEL = 3   # BGR888


# ==============================================
# 유틸 함수: RGB → BGR888 변환
# ==============================================
def convert_to_bgr888(image_path):
    img = Image.open(image_path).convert("RGB")
    img = img.resize((WIDTH, HEIGHT))
    rgb = np.array(img)
    bgr = rgb[:, :, ::-1]  # RGB → BGR
    return bgr.astype(np.uint8).tobytes()


# ==============================================
# 1) 폴더 내 이미지 스캔
# ==============================================
image_files = sorted([
    f for f in os.listdir(IMAGE_FOLDER)
    if f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp"))
])

print("Found images:", image_files)

# ==============================================
# 2) 이미지 변환 및 big.bin 생성
# ==============================================
offset_table = []
current_offset = 0

with open(OUTPUT_BIN, "wb") as bin_out:
    for name in image_files:
        full_path = os.path.join(IMAGE_FOLDER, name)

        # BGR888 변환
        bgr_data = convert_to_bgr888(full_path)

        # Offset 기록
        offset_table.append((name, BASE_ADDR + current_offset))

        # BIN에 추가
        bin_out.write(bgr_data)

        # 다음 offset 갱신
        current_offset += WIDTH * HEIGHT * BYTES_PER_PIXEL


# ==============================================
# 3) .h 파일 생성
# ==============================================
with open(OUTPUT_H, "w") as h:
    h.write("// Auto-generated Image Index Table\n\n")
    h.write(f"#define IMAGE_WIDTH   {WIDTH}\n")
    h.write(f"#define IMAGE_HEIGHT  {HEIGHT}\n")
    h.write(f"#define IMAGE_BPP     {BYTES_PER_PIXEL}\n")
    h.write(f"#define IMAGE_SIZE    (IMAGE_WIDTH * IMAGE_HEIGHT * IMAGE_BPP)\n\n")
    h.write(f"#define IMAGE_BASE_ADDR  0x{BASE_ADDR:08X}\n\n")

    h.write(f"#define IMAGE_COUNT {len(image_files)}\n\n")

    h.write("typedef struct {\n")
    h.write("    const char *name;\n")
    h.write("    uint32_t offset;\n")
    h.write("} image_entry_t;\n\n")

    h.write("static const image_entry_t image_table[] = {\n")
    for name, offset in offset_table:
        h.write(f'    {{"{name}", 0x{offset:08X}}},\n')
    h.write("};\n\n")

print("DONE!")
print("Generated:", OUTPUT_BIN, OUTPUT_H)


Found images: ['EggFried.jpg', 'Meat.jpg', 'PASTA.jpg', 'RICE.jpg', 'Seafood.jpg', 'hamburg.jpg', 'pizza.jpg', 'tomato.jpg']
DONE!
Generated: images_bgr888_concat.bin image_index_table.h
